In [1]:
# prerequisite
# pip install -U imbalanced-learn

In [2]:
from pipelines.DataPipeline import DataPipeline
from transformers.Preprocessing import Preprocessing
from sklearn.model_selection import train_test_split
from utils.Common import Config
from imblearn.over_sampling import SMOTE

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

import pandas as pd
import numpy as np
import joblib

In [3]:
RAW_DATA_PATH = "../data/raw/KSI.csv"
df = pd.read_csv(RAW_DATA_PATH)

In [4]:
# fill missing values, adding new columns, extracting useful columns
pc = Preprocessing(df, Config.binary_columns, Config.cat_attribs, Config.num_attribs, Config.label)
new_df = pc.getFrame()

In [5]:
# seperate feature and label
X = new_df[Config.cat_attribs + Config.num_attribs+ Config.binary_columns]
Y = new_df[Config.label]

In [6]:
# pass feature to pipeline and convert it to numerical data
dp = DataPipeline(Config.num_attribs,Config.cat_attribs)
X = dp.process(X)

In [7]:
X.isna().sum().sum()

0

In [8]:
Y.value_counts().tolist()

[14246, 2201]

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=Config.test_size, stratify=Y)

In [10]:
smote_minority = SMOTE(n_jobs = -1, sampling_strategy = "minority")
X_train_sm, y_train_sm = smote_minority.fit_resample(X_train, y_train)


c:\Users\yvonn\anaconda3\lib\site-packages\imblearn\over_sampling\_smote\base.py:336: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(


In [12]:

# Create param grid



param_grid = {
    "learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    "min_samples_split": np.linspace(0.5, 1, 5),
    #"min_samples_leaf": np.linspace(0.1, 0.5, 12),
    "max_depth":[3,5,8],
    "max_features":["sqrt"],
    "criterion": ["friedman_mse",  "squared_error"],
    "subsample":[0.8],
    "n_estimators":[10]
    }

clf = GridSearchCV(estimator=GradientBoostingClassifier(),param_grid=param_grid,n_jobs=10,cv=10)
best_clf = clf.fit(X_train_sm, y_train_sm)
best_clf.best_estimator_
best_clf.best_score_


0.7215333941479412

In [ ]:
best_model = best_clf.best_estimator_
best_model.score(X_test,y_test)

0.8036474164133739

In [ ]:
best_clf.best_params_
# {'C': 100, 'penalty': 'l2', 'solver': 'saga'}

{'criterion': 'friedman_mse',
 'learning_rate': 0.2,
 'max_depth': 8,
 'max_features': 'sqrt',
 'n_estimators': 10,
 'subsample': 0.95}

In [11]:
import joblib

best_model = GradientBoostingClassifier(criterion='friedman_mse',  learning_rate=0.2,  max_depth=8,  max_features='sqrt',  n_estimators=10,  subsample=0.95)
best_model.fit(X_train_sm, y_train_sm)

joblib.dump(best_model,'../models/best_model_gradientboost.pkl')


['../models/best_model_gradientboost.pkl']